# Live GPS Plotting from NMEA TCP Socket

This notebook connects to a local TCP socket streaming NMEA sentences from a GPS module, parses the data, and plots the live GPS track on an interactive map.

In [ ]:
from pynmeagps import NMEAMessage, haversine


def parse_nmea_latlon(msg: NMEAMessage):
    if msg.msgID == 'GGA' and  msg.lat != '':
        return (msg.lat, msg.lon)
    return None

def calc_distance_m(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
    '''Calculates distance in meters using haversine formula'''
    return haversine(lat1, lon1, lat2, lon2) * 1000


In [ ]:
import socket
import time

from ipyleaflet import CircleMarker, LayerGroup, Map, Polyline, DivIcon, Marker
from pynmeagps import NMEAReader, NMEAMessage
from IPython.display import display

# Configuration
TCP_HOST = 'localhost'
TCP_PORT = 12345  # Change to your NMEA TCP port

# Map setup
INIT_CENTER = (51.0, 10.0)  # Default center (Germany)
m = Map(center=INIT_CENTER, zoom=14)
track_layer = LayerGroup()
m.add(track_layer)
m.layout.height = '600px'
display(m)

# Store all points for the polyline
track_points = []

# Store latest marker (as a Circle)
latest_circle = None

poly = None
first_point = True

sock = None
speed_marker = None
speed = 0
last_pos = None
last_time = 0
try:
    sock = socket.create_connection((TCP_HOST, TCP_PORT))
    sock_file = sock.makefile('r', encoding='ascii')
    while len(line := sock_file.readline()) > 0:
        nmea_str = line.strip()
        nmea: NMEAMessage = NMEAReader.parse(nmea_str)
        if nmea.msgID == 'GGA' and nmea.quality == 1:
            pos = (nmea.lat, nmea.lon)
            track_points.append(pos)

            # Remove previous circle marker
            if latest_circle:
                track_layer.remove(latest_circle)
            if speed_marker:
                track_layer.remove(speed_marker)

            current_time = time.time()
            if last_pos is not None:
                dist = calc_distance_m(last_pos[0], last_pos[1], pos[0], pos[1])
                print(dist)
                speed = dist / (current_time - last_time) * 3.6
            print(speed)
            last_time = current_time
            last_pos = pos

            # Add a new circle marker for the latest point
            text_icon = DivIcon(
                html=f'<div style="font-size:12px; background:white; padding:2px 4px; border-radius:4px;">{speed:.1f} km/h</div>',
                icon_size=(100, 30),
                icon_anchor=(0, -15)  # adjust to position text above the circle
            )
            speed_marker = Marker(location=pos, icon=text_icon)
            latest_circle = CircleMarker(location=pos, radius=4, color='blue', fill_color='blue', fill_opacity=0.8, weight=2)
            track_layer.add(latest_circle)
            track_layer.add(speed_marker)

            # Remove previous polyline
            if poly:
                track_layer.remove(poly)

            # Draw polyline for the whole track
            poly = Polyline(locations=track_points, color='red', fill=False, weight=2)
            track_layer.add(poly)

            # Center map only on first datapoint
            if first_point:
                m.center = pos
                first_point = False

except KeyboardInterrupt:
    print('Stopped by user.')
finally:
    if sock:
        try:
            sock.shutdown(socket.SHUT_RDWR)
        except Exception:
            pass
        sock.close()
        print('Socket closed.')

In [ ]:
# Plot GPS track from a log file (NMEA sentences, one per line)
from datetime import datetime

from ipyleaflet import CircleMarker, LayerGroup, Map, Polyline
from IPython.display import display
from pynmeagps import NMEAReader, NMEAMessage

LOG_FILE = '/home/florian/data/recordings/WAS/2025-07-17_14-10-32_gps.log'  # Change to your log file path

track_points = []
with open(LOG_FILE, 'r', encoding='ascii') as f:
    for line in f:
        iso_str, nmea_str = line.strip().split(';')
        timestamp = datetime.fromisoformat(iso_str).timestamp()
        nmea: NMEAMessage = NMEAReader.parse(nmea_str)
        if nmea.msgID == 'GGA' and nmea.quality == 1:
            pos = (nmea.lat, nmea.lon)
            track_points.append(pos)

if track_points:
    INIT_CENTER = track_points[0]
    m = Map(center=INIT_CENTER, zoom=14)
    track_layer = LayerGroup()
    m.add(track_layer)
    m.layout.height = '600px'
    poly = Polyline(locations=track_points, color='red', fill=False, weight=2)
    track_layer.add(poly)
    latest_circle = CircleMarker(location=track_points[-1], radius=4, color='blue', fill_color='blue', fill_opacity=0.8, weight=2)
    track_layer.add(latest_circle)
    display(m)
else:
    print('No valid points found in log file.')